# Задание 1

In [4]:
def sum_distance(f, t):
    if f > t:
        f, t = t, f
    sum = 0
    for i in range(f, t + 1):
        sum += i
    return sum

In [7]:
f = int(input())
t = int(input())
print(sum_distance(f, t))

45


# Задание 2

In [9]:
def trim_and_repeat(str, offset = 0, repetitions = 1):
    return str[offset:] * repetitions

In [ ]:
str = input()
offset = int(input())
repetitions = int(input())

print(trim_and_repeat(str, offset, repetitions))

cd
